In [162]:
import spacy
import pandas as pd
import numpy as np
import torch
from tqdm import tqdm
from spacy.tokens import DocBin
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.version.cuda)
import huggingface_hub
import chardet
#import os
#os.environ['KMP_DUPLICATE_LIB_OK']='True'

2.0.1+cu118
True
11.8


In [176]:
nlp = spacy.load("en_core_web_sm")
with open("C:/Users/fedor/Documents/NLP/task1/tripadvisor_hotel_reviews.csv", 'rb') as f:
    result = chardet.detect(f.read())
    
print(result['encoding'])

utf-8


In [182]:
df = pd.read_csv("C:/Users/fedor/Documents/NLP/task1/tripadvisor_hotel_reviews.csv", encoding=result['encoding'], sep=',')
df = df.sort_values(by=['Rating'], ascending=False)
df

,Review,Rating
10245,"best deal new orleans hotel wonderful bargain,...",5
7236,family france traveled paris husband 3 childre...,5
14946,great stay jazz stayed hotel jazz 15th decembe...,5
7228,quiet great location lovely hotel long weekend...,5
14945,"quiet spent nights thoroughly enjoyed, hotel i...",5
...,...,...
15106,holiday inn express better holiday inn hit gym...,1
10527,not 3 star maybe 1 star say location wise good...,1
1356,look shabby large hotel absolutely no room ser...,1
7030,"secrets excellence, sorry nothing excellent re...",1


In [183]:
df = df[['Review', 'Rating']]
df = df.sort_values(by=['Rating'], ascending=False)
df

,Review,Rating
10245,"best deal new orleans hotel wonderful bargain,...",5
2570,expectations bought stay hotel charity auction...,5
2562,"great place, time greeted resort staff airport...",5
18495,good times majestic colonial just returned wee...,5
18490,"great all-inclusive, beautiful weather, soft w...",5
...,...,...
15823,overcharge great hotel great room amenities gy...,1
349,"n't bother dump, door room warped room hallway...",1
6118,lesson learned air conditioners dont 78 degree...,1
6156,elevator horror husband stayed 6 days 2 days e...,1


In [184]:
df["Rating"].value_counts()

Rating
5    9054
4    6039
3    2184
2    1793
1    1421
Name: count, dtype: int64

In [185]:
df = df.replace({'Rating': {1 : 0, 2 : 0, 3 : 0}})
df = df.replace({'Rating': {5 : 1, 4 : 1}})
df["Rating"].value_counts()

Rating
1    15093
0     5398
Name: count, dtype: int64

In [186]:
df

,Review,Rating
10245,"best deal new orleans hotel wonderful bargain,...",1
2570,expectations bought stay hotel charity auction...,1
2562,"great place, time greeted resort staff airport...",1
18495,good times majestic colonial just returned wee...,1
18490,"great all-inclusive, beautiful weather, soft w...",1
...,...,...
15823,overcharge great hotel great room amenities gy...,0
349,"n't bother dump, door room warped room hallway...",0
6118,lesson learned air conditioners dont 78 degree...,0
6156,elevator horror husband stayed 6 days 2 days e...,0


In [187]:
df = df.sample(frac=1)

In [191]:
df

,Review,Rating
7915,perfect location family vist chinese new year ...,1
11489,"live, great rooms excellent service wonderful ...",1
14293,brilliant little discovery heart madrid concur...,1
4458,"friendly daughter stayed 1 night hotel, rooms ...",1
4898,"delightful stay, teenage daughters just return...",1
...,...,...
9202,excellent choice chose stay hotel good reviews...,1
20047,"sun sand smiles people, want 5 star pay 4 star...",1
18523,"boyfriend cant wait, majestic colonial best re...",1
17130,"recommend place heart beat, just got week bahi...",1


In [192]:
data = [tuple(df.iloc[i].values) for i in range(df.shape[0])]

In [201]:
train_data = data[5000:]
valid_data = data[:5000]
print(len(train_data), len(valid_data))
valid_data

15491 5000


[('perfect location family vist chinese new year thoroughly pleased stay omni, adjoining rooms king double assistd turn omnii staff, stayed dozen hotels s.f, st. francis palace milano hyatts mariott sir francis drake kensington park, years definately best, hotels nice lobbies ordinary rooms, omni beautifully furnished luxurious rooms imaginable amenitiy, recommend, san francisco developing east union square restaurants shopping attractions near financial district location omni, plus california cable car line right outside, s.f, chinese new year parade block away prom dress shopping 5 min walk attend event moma 10 minute walk, nearby belden place offers san franciscos best restaurants truly european bistro experience entirely family friendly, dinner brindisi, make reservations weekends s.f, restaurant.. hotel valet parking 45 high san francisco, corner kearny 24 hour city parking lot security, bargain, join omni club stay real pampering including morning coffee delivered door paper free

In [198]:
def make_docs(data):
    """
    this will take a list of texts and labels
    and transform them in spacy documents
    data: list(tuple(text, label))
    returns: List(spacy.Doc.doc)
    """
    docs = []
    # nlp.pipe([texts]) is way faster than running
    # nlp(text) for each text
    # as_tuples allows us to pass in a tuple,
    # the first one is treated as text
    # the second one will get returned as it is.
    # a = tqdm(nlp.pipe(data, as_tuples=True), total = len(data))
    for doc, label in tqdm(nlp.pipe(data, as_tuples=True), total = len(data)):
        #print(label)
        if label == 0:
            print('-')
            doc.cats['1'] = 0
            doc.cats['0'] = 1
        else:
            print('+')
            doc.cats['1'] = 1
            doc.cats['0'] = 0
        # we need to set the (text)cat(egory) for each document
        #doc.cats["positive"] = label
        # put them into a nice list
        docs.append(doc)
    return docs

In [199]:
num_texts = 5000
train_docs = make_docs(train_data)

100%|███████████████████████████████████████████████████████████████████████████| 15491/15491 [01:50<00:00, 139.72it/s]


In [200]:
num_texts = 1000
# then we save it in a binary file to disc
doc_bin = DocBin(docs=train_docs)
doc_bin.to_disk("./train.spacy")
# repeat for validation data
valid_docs = make_docs(valid_data)
doc_bin = DocBin(docs=valid_docs)
doc_bin.to_disk("./dev.spacy")

100%|█████████████████████████████████████████████████████████████████████████████| 5000/5000 [00:35<00:00, 140.78it/s]


In [202]:
!python -m spacy init fill-config ./base_config.cfg ./config.cfg

[+] Auto-filled config with all values
[+] Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
!python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy --output ./output

In [130]:
import spacy
# load thebest model from training
nlp = spacy.load("output/model-best")
text = ""
print("type : ‘quit’ to exit")
# predict the sentiment until someone writes quit
while text != "quit":
    text = input("Please enter example input: ")
    doc = nlp(text)
    print(doc.cats)
    if doc.cats['1'] >.5:
        print(f"the sentiment is positive")
    else:
        print(f"the sentiment is negative")

type : ‘quit’ to exit
Please enter example input: fuck
{'1': 1.0, '0': 9.346819318523103e-09}
the sentiment is positive
Please enter example input: no
{'1': 0.9999985694885254, '0': 1.41757516303187e-06}
the sentiment is positive
[i] Saving to output directory: output
[i] Using CPU

=========================== Initializing pipeline ===========================
[+] Initialized pipeline

============================= Training pipeline =============================
[i] Pipeline: ['transformer', 'textcat']
[i] Initial learn rate: 0.0
E    #       LOSS TRANS...  LOSS TEXTCAT  CATS_SCORE  SCORE 
---  ------  -------------  ------------  ----------  ------
  0       0           0.00          0.75       50.00    0.50
  2     200           0.23         60.20       50.00    0.50
  4     400           0.00          0.00       50.00    0.50
  7     600           0.00          0.00       50.00    0.50
  9     800           0.00          0.00       50.00    0.50
[+] Saved pipeline to output directory

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Token indices sequence length is longer than the specified maximum sequence length for this model (822 > 512). Running this sequence through the model will result in indexing errors


KeyboardInterrupt: Interrupted by user